In [1]:
from datagen.data_utils import *
import numpy as np
import sapien.core as sapien
from PIL import Image, ImageColor

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
urdf = 'data_base/laptop/10211/mobility.urdf'

camera, asset, scene = scene_setup(urdf_file=urdf)
pose = np.array([
            [
                -0.06419897079467773,
                0.7324758172035217,
                -0.6777592897415161,
                -3.0862629413604736
            ],
            [
                -0.9979371428489685,
                -0.047121405601501465,
                0.043601423501968384,
                0.19854456186294556
            ],
            [
                0.0,
                0.6791603565216064,
                0.733989953994751,
                3.3423168659210205
            ],
            [
                0.0,
                0.0,
                0.0,
                1.0
            ]
        ])

[2023-09-29 10:33:01.352] [SAPIEN] [warning] A second engine will share the same internal structures with the first one. Arguments passed to constructor will be ignored.
[2023-09-29 10:33:01.352] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.
[2023-09-29 10:33:01.352] [SAPIEN] [warning] Setting renderer more than once should be avoided.


In [3]:
pose.shape


(4, 4)

In [4]:
from pathlib import Path as P
save_dir = P('./draft/view_test')
save_dir.mkdir(exist_ok=True)

# render_img_with_pose(pose, save_dir, None, scene, camera, asset)
point = random_point_in_sphere(5, theta_range=[1 * math.pi, 1.5*math.pi], phi_range=[0.4*math.pi, 0.5*math.pi])
mat44 = calculate_cam_ext(point)
camera.set_pose(sapien.Pose.from_transformation_matrix(mat44))

scene.step()  # make everything set
scene.update_render()
camera.take_picture()
rgba = camera.get_float_texture('Color')  # [H, W, 4]
        
rgba_img = (rgba * 255).clip(0, 255).astype("uint8")
rgba_pil = Image.fromarray(rgba_img, 'RGBA')
rgba_pil.save('./draft/test_view.png')

In [5]:
joint_list = asset.get_joints()
r_joint = joint_list[-1]


In [6]:
asset.get_qpos()

array([0.], dtype=float32)

In [7]:
def degrees_to_radians(degrees):
    radians = degrees * (math.pi / 180)
    return radians

asset.set_qpos(degrees_to_radians(15))
scene.step()  # make everything set
scene.update_render()
camera.take_picture()
rgba = camera.get_float_texture('Color')  # [H, W, 4]
        
rgba_img = (rgba * 255).clip(0, 255).astype("uint8")
rgba_15_pil = Image.fromarray(rgba_img, 'RGBA')
rgba_15_pil.save('./draft/test_view_15.png')

In [8]:
# joint_state = r_joint.get_pose_in_parent().to_transformation_matrix()
joint_state = asset.get_links()[-1].pose.to_transformation_matrix()

In [9]:
def transform_V1_to_V2(joint_state, V1, delta_rotation):
    try:
        # Extract the rotation matrix R_joint from the joint state
        R_joint = joint_state[:3, :3]

        u = R_joint[0, 2]
        v = R_joint[1, 2]
        w = R_joint[2, 2]


        # Calculate the rotation matrix R_theta for the delta rotation along the joint axis
        cos_theta = np.cos(delta_rotation)
        sin_theta = np.sin(delta_rotation)
        R_theta = np.array([[cos_theta + u**2 * (1 - cos_theta), u * v * (1 - cos_theta) - w * sin_theta, u * w * (1 - cos_theta) + v * sin_theta],
                            [v * u * (1 - cos_theta) + w * sin_theta, cos_theta + v**2 * (1 - cos_theta), v * w * (1 - cos_theta) - u * sin_theta],
                            [w * u * (1 - cos_theta) - v * sin_theta, w * v * (1 - cos_theta) + u * sin_theta, cos_theta + w**2 * (1 - cos_theta)]])


        # Create 4x4 transformation matrices
        T_joint = np.eye(4)
        T_joint[:3, :3] = R_joint

        T_theta = np.eye(4)
        T_theta[:3, :3] = R_theta

        # Compute V2 by multiplying the transformation matrices
        V2 = np.dot(np.dot(np.dot(T_joint, T_theta), np.linalg.inv(T_joint)), V1)

        return V2

    except Exception as e:
        print(f"Error calculating V2: {str(e)}")
        return None

In [10]:
v2 = transform_V1_to_V2(joint_state, mat44, degrees_to_radians(15))

In [11]:
asset.set_qpos(0)
camera.set_pose(sapien.Pose.from_transformation_matrix(v2))

scene.step()  # make everything set
scene.update_render()
camera.take_picture()
rgba = camera.get_float_texture('Color')  # [H, W, 4]
        
rgba_img = (rgba * 255).clip(0, 255).astype("uint8")
rgba_v2_pil = Image.fromarray(rgba_img, 'RGBA')
rgba_v2_pil.save('./draft/test_view_15_v2.png')

In [12]:
asset.get_links()[-1].pose.to_transformation_matrix()

array([[-1.1920929e-07,  2.9802322e-08, -1.0000000e+00,  2.4714707e-01],
       [-1.0000000e+00,  0.0000000e+00,  1.1920929e-07,  0.0000000e+00],
       [-2.9802322e-08,  1.0000000e+00,  5.9604645e-08, -7.7060461e-03],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32)

In [13]:
link = asset.get_links()[-1]

In [14]:
from utils.visualization import overlay_images
overlayed = overlay_images(rgba_15_pil, rgba_v2_pil, 0.5)
overlayed.save('./draft/overlayed_view.png')

In [15]:
overlayed_base = overlay_images(rgba_pil, rgba_15_pil, 0.5)
overlayed_base.save('./draft/overlayed_art.png')

In [16]:
links = asset.get_links()

In [17]:
links

[Actor(name="base", id="1"),
 Actor(name="link_0", id="2"),
 Actor(name="link_1", id="3")]

In [23]:
import json

def load_json_to_dict(fname):
    try:
        with open(fname, 'r') as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"Error loading JSON from {fname}: {str(e)}")
        return None

In [30]:
json_fname = './data_base/laptop/10211/mobility_v2.json'
meta = load_json_to_dict(json_fname)

In [36]:
asset.get_links()

[Actor(name="base", id="1"),
 Actor(name="link_0", id="2"),
 Actor(name="link_1", id="3")]

In [37]:
meta

[{'id': 0,
  'parent': -1,
  'joint': 'junk',
  'name': 'other_leaf',
  'parts': [{'id': 12, 'name': 'other_leaf', 'children': []}],
  'jointData': {}},
 {'id': 1,
  'parent': -1,
  'joint': 'free',
  'name': 'base_side',
  'parts': [{'id': 7, 'name': 'keyboard', 'children': []},
   {'id': 17, 'name': 'touchpad_surface', 'children': []},
   {'id': 9, 'name': 'base_frame', 'children': []}],
  'jointData': {}},
 {'id': 2,
  'parent': -1,
  'joint': 'junk',
  'name': 'other',
  'parts': [{'id': 5, 'name': 'other_leaf', 'children': []},
   {'id': 14, 'name': 'other_leaf', 'children': []}],
  'jointData': {}},
 {'id': 3,
  'parent': 1,
  'joint': 'hinge',
  'name': 'shaft',
  'parts': [{'id': 6, 'name': 'shaft', 'children': []},
   {'id': 15, 'name': 'screen', 'children': []},
   {'id': 16, 'name': 'screen_frame', 'children': []}],
  'jointData': {'axis': {'origin': [0,
     -0.007706040424053753,
     -0.24714714808389615],
    'direction': [1, 0, 0]},
   'limit': {'a': 105.11999999999999,

In [40]:
link.type

for link in links:
    print(link.type)

kinematic_link
kinematic_link
kinematic_link


In [43]:
test = asset.get_joints()[-1]

In [44]:
test.__dir__()

['__init__',
 '__doc__',
 '__module__',
 'name',
 'type',
 '__repr__ ',
 'get_name',
 'set_name',
 'get_parent_link',
 'get_child_link',
 'get_dof',
 'get_pose_in_parent',
 'get_pose_in_child',
 'get_limits',
 'set_limits',
 'articulation',
 '__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [45]:
test.get_parent_link()

Actor(name="link_0", id="2")

In [46]:
test.get_child_link()

Actor(name="link_1", id="3")

In [49]:
links[-1].get_pose()

Pose([0.247147, 0, -0.00770605], [0.5, 0.5, -0.5, -0.5])